In [1]:
"""you need to have patchify and cv2"""
import os
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from patchify import patchify
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
def extract_patches(image_list, mask_src, image_src, mask_dst, image_dst, patch_size):
    
    for im in tqdm(image_list):
        """ read image and mask then extract patches and diregard patches with no label"""
        img = cv2.imread(os.path.join(image_src, im))
        msk = cv2.imread(os.path.join(mask_src, im),0)
        img_patches = patchify(img, (patch_size, patch_size,3), step=patch_size)
        mask_patches = patchify(msk, (patch_size, patch_size), step=patch_size) 
        img_patches =img_patches.reshape((-1, patch_size, patch_size,3))
        mask_patches =mask_patches.reshape((-1, patch_size, patch_size))
        #Step=256 for patch_sie patches means no overlap
        for i in range(img_patches.shape[0]):
            mask_patch = mask_patches[i]
            outside = np.all(mask_patch == 0)
            if not outside:
                img_patch = img_patches[i]
                filename = im.split(".png")[0] + "_" + str(i) + ".png"
#                 mask_file= os.path.join(mask_src, filename)
                cv2.imwrite(os.path.join(image_dst, filename), img_patch)
                cv2.imwrite(os.path.join(mask_dst, filename), mask_patch)



In [3]:
df =pd.read_csv('pathdata.csv')
train_df, test_df= train_test_split(df, test_size=0.2, random_state=42)

In [4]:
df.shape

(151, 4)

In [5]:
df.head(3)

,images_path,height,width,channels
0,TCGA-E9-A5FL-DX1_xmin45595_ymin47283_MPP-0.230...,12622,6607,3
1,TCGA-OL-A6VO-DX1_xmin88416_ymin188485_MPP-0.23...,4492,4364,3
2,TCGA-AR-A5QQ-DX1_xmin107130_ymin49863_MPP-0.23...,5814,5724,3


In [ ]:
image_path='./data_dir/images/'
mask_path = './data_dir/masks/'
image_list=train_df.images_path.values.tolist()
mask_src=mask_path
image_src = image_path
mask_dst= 'Patches/Largetrain/masks/'
image_dst ='Patches/Largetrain/images/'
patch_size =1024

In [ ]:
extract_patches(image_list, mask_src, image_src, mask_dst, image_dst, patch_size)

In [ ]:
image_list=test_df.images_path.values.tolist()
mask_src=mask_path
image_src = image_path
mask_dst= 'Patches/Largetest/masks/'
image_dst ='Patches/Largetest/images/'
patch_size =1024

In [ ]:
extract_patches(image_list, mask_src, image_src, mask_dst, image_dst, patch_size)